In [242]:
import vtk
import plotly.graph_objs as go
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [243]:
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("mixture.vti")
reader.Update()
#Reading th input 3D data file using vtk
data = reader.GetOutput()


In [244]:
dimensions = data.GetDimensions()
scalar_range = data.GetScalarRange()

data_array = np.array(data.GetPointData().GetArray(0)).reshape(dimensions, order='F')
print(scalar_range)

(-0.9935540556907654, 0.43280163407325745)


In [245]:
x, y, z = np.mgrid[0:dimensions[0], 0:dimensions[1], 0:dimensions[2]]

# Flatten the arrays to get the coordinates of each point
x = x.flatten()
y = y.flatten()
z = z.flatten()

# Flatten the array of scalar values to get the value of each point
values = data_array.flatten()

print(x.shape,y.shape,z.shape,values.shape)

(421875,) (421875,) (421875,) (421875,)


In [246]:
slide_value = widgets.FloatSlider(          # Creating slider for interactive interface
    value=0.0,
    min=-0.9935540556907654,
    max=0.43280163407325745,
    step=0.1,
    description="Isovalue",
    orientation="horizontal",
    continuous_update = False
)

button = widgets.Button(description="Reset") # Creating Reset Button 
output = widgets.Output()
inp = widgets.HBox([slide_value])
display(button, output)
layout = go.Layout(title='Histogram', xaxis=dict(title='Vortex scalar values'), yaxis=dict(title='Frequency'))
iso_layout = go.Layout(title='Isosurface')
# Extracting Isosurface using plotly and jupyter widgets
iso=go.FigureWidget(data = go.Isosurface(
    x=x,
    y=y,
    z=z,
    value=values,
    isomin=0.0,
    isomax=0.0,
    surface_count=1,
    colorscale='Plasma',
    cmin = -0.99,
    cmax = 0.43,
    opacity=0.6
    ),layout = iso_layout)

# Plotting Histogram plot using plotly and jupyter widgets
hgm = go.FigureWidget(data = [go.Histogram(x = values,nbinsx=30)],layout=layout)
def update_isosurface(isovalue=0.0):
    iso.data[0].isomin = isovalue
    iso.data[0].isomax = isovalue
    filtered_values = []
    for i in values:
        if((i >= (isovalue - 0.25)) and (i <= (isovalue + 0.25))):
            filtered_values.append(i)
    if isovalue == 0.0:
        hgm.data[0].x = values
    else:       
        hgm.data[0].x = filtered_values
    display(widgets.HBox([iso,hgm]))
    
def on_button_clicked(b):
    with output:
        iso.data[0].isomin = 0.0
        iso.data[0].isomax = 0.0
        hgm.data[0].x = values
        slide_value.value = 0.0
        
button.on_click(on_button_clicked) #calling function for response on button click

out = widgets.interactive_output(update_isosurface, {'isovalue': slide_value}) #calling function for response on slider change

display(inp,out)

Button(description='Reset', style=ButtonStyle())

Output()

Output()